In [1]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
def convert_mat_np(gender=None):
    m_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
    f_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat"
    # m_file = "../raw_data/Emotrans1_Boy_data_preprocessed_33.mat"
    # f_file = "../raw_data/Emotrans1_girl_data_preprocessed_33.mat"
    if gender == "f":
        data_dict_female = mat73.loadmat(f_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])
    elif gender == 'm':
        data_dict_female = mat73.loadmat(m_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])  
    else: 
        raise Exception("gender not defined")

In [3]:
def combine_male_female():
    m = convert_mat_np("m")
    f = convert_mat_np("f")
    return np.concatenate((m,f),axis=0)

In [4]:
# input: data and index of the removed person 
# output: data without removed person
def remove_person(df,index):
    new_df = []
    for i in range(df.shape[0]):
        if i in index:
            continue 
        new_df.append(df[i])
    return np.array(new_df)

In [5]:
# type: 0 --> average 3+4 and 1+2; 1--> use only 3 and 2  
def get_pos_or_neg(data,position,type=0):
    arr = []
    index = 0
    for i in range(data.shape[0]):
#         print(data[i].shape)
        if type == 0:
            diff = (data[i][position[0][0]][position[0][1]]+data[i][position[1][0]][position[1][1]])/2 - (data[i][position[3][0]][position[3][1]]+data[i][position[2][0]][position[2][1]])/2
        elif type == 1:
             diff = (data[i][position[1][0]][position[1][1]] - data[i][position[2][0]][position[2][1]])/2
        arr.append(diff)
    return np.array(arr)

In [6]:
# select frequenceis and (0-4s -> 0 or 0.5-4.5s -> 1)
# output: x 
def choose_freq(data=None,freq = None, sec=None):
    if freq:
        data = np.delete(data,freq,axis=3)
    if sec == 0:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 1, axis=3)
        return data
    elif sec == 1:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 0, axis=3) 
        return data
    else:
        print('sec not specified')
        return data 

In [7]:
# reshape to 128*8*2 = 2048
# return x 
def squeeze_feature_size(data):
    for d in range(df.shape[0]):
#         for i in range(df[d].shape[0]):
        size = data[d].shape
        data[d] = data[d].reshape(size[0], size[1]*size[2]*size[3])
    return data

In [8]:
# generate labels for x 
# input: data and label_type (0:zeros, 1: ones )
def generate_labels(df,type=None):
    arr = []
    label = []
    label_type = -1
    if type == 0:
        label_type = 0
    elif type == 1:
        label_type = 1
    else:
        print("Wrong type - get_pos_or_neg()")
        return 
    for d in range(df.shape[0]):
#         for i in range(df[d].shape[0]):
        size = df[d].shape
#         print(size)
        if label_type == 1:
            label.append(np.ones((size[0],1)))
        else:
            label.append(np.zeros((size[0],1)))
    return np.array(label)

In [9]:
# combine net and neg 
def combine_net_neg(netX,netY,negX,negY):
    for i in range(netX.shape[0]):
        netX[i] = np.concatenate((netX[i],negX[i]),axis=0)
        netY[i] = np.concatenate((netY[i],negY[i]),axis=0)
    return netX,netY

In [46]:
df = combine_male_female()

In [47]:
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
pos_df = get_pos_or_neg(df,pos)
neg_df = get_pos_or_neg(df,neg)

In [48]:
def print_shape(df):
    for d in range(df.shape[0]):
        print("# person: ",d+1)
        for i in range(df[d].shape[0]):
            for j in range(df[d][i].shape[0]):
                print("    ",j,df[d][i][j].shape)
        

In [64]:
lables = generate_labels(df,1)

(18, 2048)
(18, 2048)
(18, 2048)
(18, 2048)
(18, 2048)
(18, 2048)
(18, 2048)
(18, 2048)
(6, 2048)
(6, 2048)
(6, 2048)
(6, 2048)
(6, 2048)
(6, 2048)
(6, 2048)
(6, 2048)


ValueError: could not broadcast input array from shape (4,18,1) into shape (4)

In [60]:
def print_label(label):
    for i in range(label.shape[0]):
        print(label[i].shape)

In [108]:
# print(pos_df.shape)
# for d in range(pos_df.shape[0]):
#     print("# person:",d)
#     print(pos_df[d].shape)
#     for i in range(pos_df[d].shape[0]):
#         print("    ",i,pos_df[d][i].shape) # for d 
#     #     print(pos_df[0][0].shape) # for i 
# #     print(pos_df[0][0][0].shape) #

In [10]:
df = combine_male_female()
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# # remove = [1,2,3,4,5,6,7,8,9]
# df = remove_person(df,remove)
pos_df = get_pos_or_neg(df,pos)
neg_df = get_pos_or_neg(df,neg)
pos_sequeezed = squeeze_feature_size(pos_df)
neg_sequeezed = squeeze_feature_size(neg_df)
pos_labels = generate_labels(pos_sequeezed,1)
neg_labels = generate_labels(neg_sequeezed,0)
# netX,netY,negX,negY
x,y = combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

In [18]:
# input type- l,m,h ; x;y 
# output x,y 
def choose_x_y_stress(type,x,y):
    l = [1, 6, 9, 10, 12, 16, 19, 23, 26, 29, 31, 32, 33, 39] 
    m = [3, 5, 7, 11, 15, 18, 24, 28, 30, 34, 37] 
    h = [0, 2, 4, 8, 13, 14, 17, 20, 21, 22, 25, 27, 35, 36, 38]
    chosen = None 
    if type == "l":
        chosen = l 
    elif type == "m":
        chosen = m 
    elif type =="h":
        chosen = h 
    else:
        print("type error")
        return 
    new_x = []
    new_y = []
    for i in range(x.shape[0]):
        if i in chosen:
            new_x.append(x[i])
            new_y.append(y[i])
    return np.array(new_x), np.array(new_y)


In [19]:
test_x, test_Y = choose_x_y_stress("l",x,y)

In [20]:
# netX,netY,negX,negY
print(test_x.shape)
print(len([1, 6, 9, 10, 12, 16, 19, 23, 26, 29, 31, 32, 33, 39]))
# for d in range(test_x.shape[0]):
#     print("# person:",d)
#     print(test_x[d].shape,test_Y[d].shape)

(14,)
14


In [13]:
# netX,netY,negX,negY
for d in range(x.shape[0]):
    print("# person:",d)
    print(x[d].shape,y[d].shape)
    #     print(pos_df[0][0].shape) # for i 
#     print(pos_df[0][0][0].shape) #

# person: 0
(36, 2048) (36, 1)
# person: 1
(36, 2048) (36, 1)
# person: 2
(36, 2048) (36, 1)
# person: 3
(36, 2048) (36, 1)
# person: 4
(36, 2048) (36, 1)
# person: 5
(36, 2048) (36, 1)
# person: 6
(36, 2048) (36, 1)
# person: 7
(36, 2048) (36, 1)
# person: 8
(36, 2048) (36, 1)
# person: 9
(36, 2048) (36, 1)
# person: 10
(36, 2048) (36, 1)
# person: 11
(36, 2048) (36, 1)
# person: 12
(36, 2048) (36, 1)
# person: 13
(36, 2048) (36, 1)
# person: 14
(36, 2048) (36, 1)
# person: 15
(36, 2048) (36, 1)
# person: 16
(30, 2048) (30, 1)
# person: 17
(30, 2048) (30, 1)
# person: 18
(30, 2048) (30, 1)
# person: 19
(36, 2048) (36, 1)
# person: 20
(36, 2048) (36, 1)
# person: 21
(36, 2048) (36, 1)
# person: 22
(36, 2048) (36, 1)
# person: 23
(36, 2048) (36, 1)
# person: 24
(36, 2048) (36, 1)
# person: 25
(36, 2048) (36, 1)
# person: 26
(36, 2048) (36, 1)
# person: 27
(36, 2048) (36, 1)
# person: 28
(36, 2048) (36, 1)
# person: 29
(30, 2048) (30, 1)
# person: 30
(30, 2048) (30, 1)
# person: 31
(36, 

In [109]:
rf_model = RandomForestClassifier(n_estimators=150)

In [133]:
def get_x_train_test(data,index):
    x_test = data[index]
    x_train = np.delete(data,index,axis=0)
    return x_train,x_test.reshape(1,-1)

def get_y_train_test(data,index):
    y_test = data[index]
    y_train = np.delete(data,index,axis=0)
    return y_train.astype(int).ravel(),y_test.astype(int)
model = RandomForestClassifier(n_estimators=50)

def model_train(x_train,x_test,y_train,y_test,model):
    model.fit(x_train,y_train)
    print("shape",x_test.shape)
    y_pred = model.predict(x_test)
    return y_test, y_pred
    
def with_train(x,y):
    # with_in
    all_accuracy = []
    # train a model 
    # return y_test, y_pred 
    avg_accuracy = []
    for d in range(x.shape[0]):
        # training set and test set
        accuracy =[]
        for i in range(x[d].shape[0]):
            x_train,x_test = get_x_train_test(x[d],i)
            y_train,y_test = get_y_train_test(y[d],i)
            y_test,y_pred = model_train(x_train,x_test,y_train,y_test,model)
            accuracy.append(accuracy_score(y_test,y_pred))
            print(x_train.shape,x_test.shape)
            print(y_train.shape,y_test.shape)
            break
        avg_accuracy.append(sum(accuracy)/len(accuracy))
        all_accuracy.append(accuracy)
        break 
    #     print(pos_df[0][0].shape) # for i 
#     print(pos_df[0][0][0].shape) #

shape (1, 2048)
[1]
[1]
(35, 2048) (1, 2048)
(35,) (1,)


In [131]:
model.predict(x_test)

array([1])

In [132]:
y_test

array([[1]])

In [134]:
from sklearn.metrics import accuracy_score

In [149]:
def get_x_y_train(x,y):
    new_x = x[0]
    new_y = y[0]
    for i in range(1,len(x)):
        new_x=np.append(new_x,x[i],axis=0)
        new_y=np.append(new_y,y[i],axis=0)
    print(new_x.shape)
    return new_x,new_y

for d in range(x.shape[0]):
    if d == 0:
        x_train,y_train = get_x_y_train(x[d+1:],y[d+1:])
        x_test,y_test = x[d],y[d]
    elif d == x.shape[0]-1:
        x_train,y_train = get_x_y_train(x[:d],y[:d])
        x_test,y_test = x[d],y[d]
    else:
        x_train,y_train = get_x_y_train(x[:d]+x[d+1:],y[:d]+y[d+1:])
        x_test,y_test = x[d],y[d]
    print(train_x.shape,test_x.shape)
    print(train_y.shape,test_y.shape)
    break

(1428, 2048)
(1428, 2048) (36, 2048)
(1428, 1) (36, 1)


In [150]:
test =  np.append(x[0],x[1],axis=0)

In [144]:
test.shape

(72, 2048)

In [159]:
def generate_label_stress(x):
    labels = []
    size = 40
    classes = [2, 0, 2, 1, 2, 1, 0, 1, 2, 0, 0, 1, 0, 2, 2, 1, 0, 2, 1, 0, 2, 2, 2, 0, 1, 2, 0, 2, 1, 0, 1, 0, 0, 0, 1, 2, 2, 1, 2, 0]
    for i in range(size):
        labels.append(np.full((x[i].shape[0]),classes[i]))
    return np.array(labels)

In [160]:
s = generate_label_stress(x)

In [161]:
for i in range(s.shape[0]):
    print(s[i].shape)

(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(30,)
(30,)
(30,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(30,)
(30,)
(36,)
(30,)
(36,)
(36,)
(36,)
(36,)
(24,)
(36,)
(36,)


In [152]:
import pandas as pd 

In [153]:
df = pd.read_csv('./results.csv')

In [155]:
df['stressGroup'].values

array(['H', 'L', 'H', 'M', 'H', 'M', 'L', 'M', 'H', 'L', 'L', 'M', 'L',
       'H', 'H', 'M', 'L', 'H', 'M', 'L', 'H', 'H', 'H', 'L', 'M', 'H',
       'L', 'H', 'M', 'L', 'M', 'L', 'L', 'L', 'M', 'H', 'H', 'M', 'H',
       'L', nan, nan], dtype=object)

In [211]:
l,m,h = [],[],[]
vals = df['stressGroup'].values
for i in range(len(vals)):
    if vals[i] == "H":
        h.append(i)
    elif vals[i] == "M":
        m.append(i)
    elif vals[i] == "L":
        l.append(i)
print(l,m,h)

[1, 6, 9, 10, 12, 16, 19, 23, 26, 29, 31, 32, 33, 39] [3, 5, 7, 11, 15, 18, 24, 28, 30, 34, 37] [0, 2, 4, 8, 13, 14, 17, 20, 21, 22, 25, 27, 35, 36, 38]


In [174]:
def combine_cases(data):
    new_df = []
    for i in range(data.shape[0]):
        xs = None
        init = True
        for j in range(data[i].shape[0]):
            for k in range(data[i][j].shape[0]):
                if init:
                    xs = data[i][j][k]
                else:
                     xs = np.append(xs,data[i][j][k],axis=0)
                init = False 
        new_df.append(xs)
    return np.array(new_df)
#         print(data[i].shape)
#         if type == 0:
#             diff = (data[i][position[0][0]][position[0][1]]+data[i][position[1][0]][position[1][1]])/2 - (data[i][position[3][0]][position[3][1]]+data[i][position[2][0]][position[2][1]])/2
#         elif type == 1:
#              diff = (data[i][position[1][0]][position[1][1]] - data[i][position[2][0]][position[2][1]])/2
#         arr.append(diff)
#     return np.array(arr)
        

In [179]:
df = combine_male_female()
df = combine_cases(df)
df = squeeze_feature_size(df)
labels = generate_label_stress(df)

In [181]:
for i in range(df.shape[0]):
    print(df[i].shape,labels[i].shape)

(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(156, 2048) (156,)
(156, 2048) (156,)
(156, 2048) (156,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(156, 2048) (156,)
(156, 2048) (156,)
(192, 2048) (192,)
(156, 2048) (156,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(192, 2048) (192,)
(128, 2048) (128,)
(192, 2048) (192,)
(192, 2048) (192,)


IndexError: index 40 is out of bounds for axis 0 with size 40

In [164]:
print_shape(df)

# person:  1
     0 (18, 128, 8, 2)
     1 (18, 128, 8, 2)
     2 (18, 128, 8, 2)
     3 (18, 128, 8, 2)
     0 (18, 128, 8, 2)
     1 (18, 128, 8, 2)
     2 (18, 128, 8, 2)
     3 (18, 128, 8, 2)
     0 (6, 128, 8, 2)
     1 (6, 128, 8, 2)
     2 (6, 128, 8, 2)
     3 (6, 128, 8, 2)
     0 (6, 128, 8, 2)
     1 (6, 128, 8, 2)
     2 (6, 128, 8, 2)
     3 (6, 128, 8, 2)
# person:  2
     0 (18, 128, 8, 2)
     1 (18, 128, 8, 2)
     2 (18, 128, 8, 2)
     3 (18, 128, 8, 2)
     0 (18, 128, 8, 2)
     1 (18, 128, 8, 2)
     2 (18, 128, 8, 2)
     3 (18, 128, 8, 2)
     0 (6, 128, 8, 2)
     1 (6, 128, 8, 2)
     2 (6, 128, 8, 2)
     3 (6, 128, 8, 2)
     0 (6, 128, 8, 2)
     1 (6, 128, 8, 2)
     2 (6, 128, 8, 2)
     3 (6, 128, 8, 2)
# person:  3
     0 (18, 128, 8, 2)
     1 (18, 128, 8, 2)
     2 (18, 128, 8, 2)
     3 (18, 128, 8, 2)
     0 (18, 128, 8, 2)
     1 (18, 128, 8, 2)
     2 (18, 128, 8, 2)
     3 (18, 128, 8, 2)
     0 (6, 128, 8, 2)
     1 (6, 128, 8, 2)
     2 (6, 12

In [182]:
df = pd.read_csv('./results.csv')

In [207]:
ans = [0 for _ in range(42)]
avg = [0.7222222222222222, 0.3333333333333333, 0.3333333333333333, 0.027777777777777776, 0.19444444444444445, 0.1111111111111111, 0.6666666666666666, 0.0, 1.0, 0.19444444444444445, 0.1111111111111111, 0.0, 0.3055555555555556, 0.0, 0.19444444444444445, 0.05555555555555555, 0.1, 0.6333333333333333, 0.4, 0.7222222222222222, 0.6944444444444444, 0.6666666666666666, 0.4166666666666667, 0.8888888888888888, 0.027777777777777776, 0.5, 0.1111111111111111, 0.4166666666666667, 0.027777777777777776, 0.6666666666666666, 0.0, 0.5555555555555556, 0.1, 0.1388888888888889, 0.6666666666666666, 0.3611111111111111, 0.6944444444444444, 0.20833333333333334, 0.7777777777777778, 0.3055555555555556]
avg+= [0,0]
for i in range(len(avg)):
    avg[i] = round(avg[i],2)
j=0
# for i in range(len(ans)):
#     if i in [11,18,36]:
#         continue 
#     ans[i] = avg[j]
#     j+=1
ans = avg
print(ans.index(min(ans)))
print(ans.index(max(ans)))
df.columns 

7
8


Index(['Exp_ID', 'exp_time', 'trais no.', 'note', 'signal_condition', 'gender',
       'age', 'race', 'handiness', 'sleep condition', 'Q11', 'Q12', 'Q13',
       'Q14', 'ID', 'ID_boy', 'ID_girl', 'within_classification', 'stress',
       'stressGroup', '42set_ID', 'between_classification_all',
       'between_stressGroup', '39set_ID', 'between_classification',
       'between_stressGroup.1', '33set_ID', 'between_classification.1',
       'between_stressGroup.2'],
      dtype='object')

In [208]:
# print(df['between_classification'])
df['between_stressGroup'] = ans
df.to_csv("restuls1.csv")

In [73]:
l = [1, 6, 9, 10, 12, 16, 19, 23, 26, 29, 31, 32, 33, 39] 
m = [3, 5, 7 , 11, 18,15, 24, 28, 30, 34, 37] # , 11, 18
h = [0, 2, 4, 8, 13, 14, 17, 20, 21, 22, 25, 27, 35,36, 38] # , 36
# h_val = [0.8055555555555556, 0.6111111111111112, 0.5833333333333334, 0.9166666666666666, 0.75, 0.7777777777777778, 0.7, 0.8055555555555556, 0.7222222222222222, 0.75, 0.8888888888888888, 0.6666666666666666, 0.7777777777777778, 0.6388888888888888, 0.6388888888888888]
# l_val = [0.8055555555555556, 0.6388888888888888, 0.75, 0.6944444444444444, 0.8888888888888888, 0.7333333333333333, 0.8611111111111112, 0.8611111111111112, 0.6944444444444444, 0.7666666666666667, 0.8611111111111112, 0.7666666666666667, 0.5555555555555556, 0.7222222222222222]
# m_val = [0.8333333333333334, 0.6944444444444444, 0.9166666666666666, 0.6666666666666666, 0.8333333333333334, 0.7, 0.8611111111111112, 0.6944444444444444, 0.5333333333333333, 0.75, 0.75]
h_val = [0.7777777777777778, 0.7222222222222222, 0.5555555555555556, 0.8611111111111112, 0.7222222222222222, 0.8055555555555556, 0.8611111111111112, 0.6388888888888888, 0.8888888888888888, 0.8333333333333334, 0.6111111111111112, 0.7666666666666667, 0.6388888888888888, 0.75]
l_val = [0.7777777777777778, 0.6666666666666666, 0.6388888888888888, 0.8055555555555556, 0.7222222222222222, 0.7666666666666667, 0.7222222222222222, 0.7777777777777778, 0.7777777777777778, 0.8333333333333334, 0.5833333333333334, 0.5833333333333334, 0.7777777777777778]
m_val = [0.8055555555555556, 0.6666666666666666, 0.8333333333333334, 0.8611111111111112, 0.7, 0.8055555555555556, 0.6388888888888888, 0.4, 0.7, 0.7083333333333334]


# def add_index(l,val):
#     for i in range(len(val)):
#         val[i] = [l[i],val[i]]
#     return val 
# val = add_index(l,l_val)
# val +=add_index(m,m_val)
# val += add_index(h,h_val)
# print(val)
# for i in [11,18,36]:
#     val +=[[i,-1]]
# val.sort(key=lambda x: x[0])
# val = [round(i[1],2) for i in val]
# val +=[-1,-1,-1]
# val

In [74]:
print(h_val)
def get_avg(vals):
    return round(sum(vals)/len(vals),2)
print(get_avg(h_val),get_avg(m_val),get_avg(l_val))

[0.7777777777777778, 0.7222222222222222, 0.5555555555555556, 0.8611111111111112, 0.7222222222222222, 0.8055555555555556, 0.8611111111111112, 0.6388888888888888, 0.8888888888888888, 0.8333333333333334, 0.6111111111111112, 0.7666666666666667, 0.6388888888888888, 0.75]
0.75 0.71 0.73


In [52]:
import pandas as pd 

In [80]:
val = [0.8888888888888888, 0.8611111111111112, 0.75, 0.6388888888888888, 0.8888888888888888, 0.8611111111111112, 0.8333333333333334, 0.6388888888888888, 0.6944444444444444, 0.6944444444444444, 0.6111111111111112, 0.6944444444444444, 0.7777777777777778, 0.6666666666666666, 0.6388888888888888, 0.75, 0.5555555555555556, 0.6388888888888888, 0.7222222222222222, 0.7222222222222222, 0.6666666666666666, 0.8888888888888888, 0.5555555555555556, 0.7222222222222222, 0.7222222222222222, 0.9166666666666666, 0.8611111111111112, 0.6944444444444444, 0.6666666666666666, 0.5833333333333334, 0.8611111111111112, 0.7222222222222222, 0.8055555555555556]
val = [round(i,3) for i in val]
df = pd.read_csv('./results.csv')
# df['between_classification.1'][14:18] = val[11:15]
df['between_classification.1'][19:36] = val[16:33]
# df['between_classification.1'][36:38] = val[33:]
df.to_csv('result1.csv')


<ipython-input-80-ee59c6d66db7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['between_classification.1'][19:36] = val[16:33]


In [81]:
val[33:]

[]

In [82]:
len(val)

33

In [87]:
l = [1, 6, 9,16, 19, 23, 26, 29, 31, 32, 33] 
m = [3, 5, 7, 15, 24, 28, 30, 34, 37] 
h = [0, 2, 4, 8, 14, 17, 20, 21, 22, 25, 27, 35, 38]
# h_val = [0.8055555555555556, 0.6111111111111112, 0.5833333333333334, 0.9166666666666666, 0.75, 0.7777777777777778, 0.7, 0.8055555555555556, 0.7222222222222222, 0.75, 0.8888888888888888, 0.6666666666666666, 0.7777777777777778, 0.6388888888888888, 0.6388888888888888]
# l_val = [0.8055555555555556, 0.6388888888888888, 0.75, 0.6944444444444444, 0.8888888888888888, 0.7333333333333333, 0.8611111111111112, 0.8611111111111112, 0.6944444444444444, 0.7666666666666667, 0.8611111111111112, 0.7666666666666667, 0.5555555555555556, 0.7222222222222222]
# m_val = [0.8333333333333334, 0.6944444444444444, 0.9166666666666666, 0.6666666666666666, 0.8333333333333334, 0.7, 0.8611111111111112, 0.6944444444444444, 0.5333333333333333, 0.75, 0.75]
h_val = [0.8611111111111112, 0.6944444444444444, 0.8055555555555556, 0.75, 0.5555555555555556, 0.6111111111111112, 0.6944444444444444, 0.9166666666666666, 0.4722222222222222, 0.8611111111111112, 0.6388888888888888]
l_val = [0.75, 0.8611111111111112, 0.6944444444444444, 0.5, 0.75, 0.6388888888888888, 0.8611111111111112, 0.6111111111111112, 0.6944444444444444, 0.8055555555555556]
m_val = [0.6111111111111112, 0.8333333333333334, 0.6388888888888888, 0.7222222222222222, 0.6388888888888888, 0.7222222222222222, 0.6666666666666666]

def add_index(l,val):
    for i in range(len(val)):
        val[i] = [l[i],round(val[i],2)]
    return val 
val = add_index(l,l_val)
val +=add_index(m,m_val)
val += add_index(h,h_val)
print(len(val))
val.sort(key=lambda x: x[0])
# val = [round(i[1],2) for i in val]
# df = pd.DataFrame(val)
# df.to_csv("./test.csv")

28


In [88]:
val

[[0, 0.86],
 [1, 0.75],
 [2, 0.69],
 [3, 0.61],
 [4, 0.81],
 [5, 0.83],
 [6, 0.86],
 [7, 0.64],
 [8, 0.75],
 [9, 0.69],
 [14, 0.56],
 [15, 0.72],
 [16, 0.5],
 [17, 0.61],
 [19, 0.75],
 [20, 0.69],
 [21, 0.92],
 [22, 0.47],
 [23, 0.64],
 [24, 0.64],
 [25, 0.86],
 [26, 0.86],
 [27, 0.64],
 [28, 0.72],
 [29, 0.61],
 [30, 0.67],
 [31, 0.69],
 [32, 0.81]]